In [8]:
import os
import pandas as pd
import json

cwd = os.getcwd()

In [21]:
# Install and authenticate Kaggle
!pip install kaggle

# Move kaggle.json to the correct location
!mkdir -p ~/.kaggle
!cp kaggle.json ~/.kaggle/
!chmod 600 ~/.kaggle/kaggle.json

# Download the dataset
!kaggle datasets download -d proxycurl/10000-us-people-profiles

# Unzip the dataset
!unzip 10000-us-people-profiles.zip

cp: cannot stat 'kaggle.json': No such file or directory
chmod: cannot access '/root/.kaggle/kaggle.json': No such file or directory
Dataset URL: https://www.kaggle.com/datasets/proxycurl/10000-us-people-profiles
License(s): unknown
10000-us-people-profiles.zip: Skipping, found more recently modified local copy (use --force to force download)
Archive:  10000-us-people-profiles.zip
replace 10000_random_us_people_profiles.txt? [y]es, [n]o, [A]ll, [N]one, [r]ename: N


In [20]:
# Read the text file
with open(cwd + '/10000_random_us_people_profiles.txt', 'r') as file:
    data = file.read()

# Split the text into individual JSON objects
json_objects = data.split('\n')

# Initialize an empty list to store valid JSON objects
valid_json_objects = []

# Filter out non-JSON content
for obj in json_objects:
    obj = obj.strip()
    if obj:  # Check if the string is not empty
        try:
            json.loads(obj)
            valid_json_objects.append(obj)
        except json.JSONDecodeError:
            pass  # Ignore non-JSON content

# Parse each JSON object and convert it to a pandas DataFrame
dfs = []
for json_str in valid_json_objects:
    json_data = json.loads(json_str)
    df = pd.json_normalize(json_data)
    dfs.append(df)

# Concatenate all DataFrames
final_df = pd.concat(dfs, ignore_index=True)

In [10]:
final_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 10000 entries, 0 to 9999
Data columns (total 40 columns):
 #   Column                        Non-Null Count  Dtype 
---  ------                        --------------  ----- 
 0   public_identifier             10000 non-null  object
 1   profile_pic_url               10000 non-null  object
 2   background_cover_image_url    10000 non-null  object
 3   first_name                    10000 non-null  object
 4   last_name                     9999 non-null   object
 5   full_name                     10000 non-null  object
 6   occupation                    8939 non-null   object
 7   headline                      9397 non-null   object
 8   summary                       6296 non-null   object
 9   country                       10000 non-null  object
 10  country_full_name             10000 non-null  object
 11  city                          9663 non-null   object
 12  state                         6687 non-null   object
 13  experiences      

In [11]:
for col in final_df.columns:
    print(final_df[col].value_counts())
    print('***'* 15)
    print()

public_identifier
aaalsaleh                   2
aamnah-khan                 2
aalokoscarmike              2
aamiramhd                   2
aadictive                   2
                           ..
aadarsh-patel               1
aadarsh-rajesh              1
aadarsh-thimma-ravikumar    1
aadarshb                    1
aaron-donald                1
Name: count, Length: 9993, dtype: int64
*********************************************

profile_pic_url
https://static-exp1.licdn.com/sc/h/244xhbkr7g40x6bsu4gi6q4ry                                                                                                                        3309
https://media-exp1.licdn.com/dms/image/C4E03AQG8fu0jtcdzag/profile-displayphoto-shrink_200_200/0/1600717537104?e=1652918400&v=beta&t=pQIP8YenDIDOsj8Fx9P77c0C9LXpMB7ebMtf17zHrfQ       2
https://media-exp1.licdn.com/dms/image/C5103AQEVsuvGP052BQ/profile-displayphoto-shrink_200_200/0/1583701548514?e=1652918400&v=beta&t=98pdFBQ1VAaNHpURia73hMGEMLwXkj_Glx7SVIve3js   

In [12]:
df_clean = final_df.iloc[:,:-8]

# Tahap 2

In [13]:
df_clean2 = df_clean[['profile_pic_url', 'full_name', 'occupation', 'headline', 'summary','country_full_name', 'city', 'state', 'experiences', 'education', 'languages']].reset_index().iloc[:,1:]

In [14]:
from datetime import date

def get_start_and_end_date(data_start, data_end):

    if data_start is not None:
        start_day_i = data_start['day']
        start_month_i = data_start['month']
        start_year_i = data_start['year']

        if data_end is None:
            end_day_i = 19
            end_month_i = 10
            end_year_i = 2023
        else:
            end_day_i = data_end['day']
            end_month_i = data_end['month']
            end_year_i = data_end['year']

        return date(start_year_i, start_month_i, start_day_i), date(end_year_i, end_month_i, end_day_i)
    else:
        return date(2000, 10, 20), date(2000, 10, 20)

def extract_data(data_dict, num_of_data, title, place):
    data_temp_years = [] ; data_temp_title = [] ; data_temp_place =[]
    if num_of_data >= 1:
        for i in range (num_of_data):
            data = data_dict[i]
            data_start = data['starts_at']
            data_end = data['ends_at']

            start_date, end_date = get_start_and_end_date(data_start, data_end)

            value_in_year = (end_date - start_date).days / 365
            data_temp_years.append(value_in_year)
            data_temp_title.append(data[title])
            data_temp_place.append(data[place])

        while len(data_temp_years) < 3:
            data_temp_years.append(None)
        while len(data_temp_title) < 3:
            data_temp_title.append(None)
        while len(data_temp_place) < 3:
            data_temp_place.append(None)

        return data_temp_years, data_temp_title, data_temp_place

    else:

        return [None, None, None], [None, None, None], [None, None, None]


In [15]:
import numpy as np

for k in range (len(df_clean2)):

    data_dict = df_clean2['experiences'][k]

    num_of_data = min(3,len(data_dict))

    data_exp_years, data_exp_title, data_exp_place = extract_data(data_dict, num_of_data, 'title', 'company')

    data_dict = df_clean2['education'][k]

    num_of_data = min(3,len(data_dict))

    data_edu_years, data_edu_title, data_edu_place = extract_data(data_dict, num_of_data, 'degree_name', 'school')

    data_edu_field = []
    if num_of_data >= 1:
        for i in range (num_of_data):
            data_edu_field.append(data_dict[i]['field_of_study'])

        while len(data_edu_field) != 3:
            data_edu_field.append(None)
    else:
        data_edu_field = [None, None, None]

    data_dict =  df_clean2['languages'][k]
    num_of_data = min(3,len(data_dict))

    data_language = []
    if num_of_data >=1  :
        data_language = list(np.copy(data_dict[0:num_of_data]))

    while len(data_language) < 3:
        data_language.append(None)

    df_i = df_clean2.iloc[k,0:7]

    df_i['1st Recent Experience in Years'], df_i['2nd Recent Experience in Years'], df_i['3rd Recent Experience in Years']   = data_exp_years
    df_i['1st Recent Experience Title'], df_i['2nd Recent Experience Title'], df_i['3rd Recent Experience Title']   = data_exp_title
    df_i['1st Recent Experience Company'], df_i['2nd Recent Experience Company'], df_i['3rd Recent Experience Company']   = data_exp_place

    df_i['1st Recent Education in Years'], df_i['2nd Recent Education in Years'], df_i['3rd Recent Education in Years']   = data_edu_years
    df_i['1st Recent Education Field of Study'], df_i['2nd Recent Education Field of Study'], df_i['3rd Recent Education Field of Study']   = data_edu_field
    df_i['1st Recent Education Title'], df_i['2nd Recent Education Title'], df_i['3rd Recent Education Title']   = data_edu_title
    df_i['1st Recent Education School'], df_i['2nd Recent Education School'], df_i['3rd Recent Education School']   = data_edu_place

    df_i['1st Language'], df_i['2nd Language'], df_i['3rd Language'] = data_language

    if k == 0:
        df_clean3 = pd.DataFrame(df_i).transpose()
    else:
        df_clean3 = df_clean3._append(df_i)

In [17]:
df_clean3.to_csv(cwd + '/Data/speaker-data.csv')